In [1]:
import pandas as pd
import geopandas as gpd
import yaml

external connectors are created by met council, using the node IDs from the first version of ranch networks. After that we modified ranch method to fix some missing links issue, so the new version of ranch network has more nodes and links, thus the IDs have been changed.  We need to first create a ID crosswalk between the versions of networks.  `shst_node_id` is stable across versions, which will be used as join key.

In [5]:
version_000_node_gdf = gpd.read_file(
    "D:/metcouncil_network_rebuild/data/processed/version_00/old/standard_networks/nodes.geojson"
)

version_001_node_gdf = gpd.read_file(
    "D:/metcouncil_network_rebuild/data/processed/version_00/standard_networks/nodes.geojson"
)

In [6]:
node_id_crosswalk_df = pd.merge(
    version_000_node_gdf[['shst_node_id', 'model_node_id']],
    version_001_node_gdf[['shst_node_id', 'model_node_id']],
    how = 'left',
    on = ['shst_node_id']
)

In [8]:
node_id_crosswalk_df.to_csv(
    "D:/metcouncil_network_rebuild/data/processed/version_00/standard_networks/new_network_node_id.csv",
    index = False
)

In [3]:
node_id_crosswalk_df = pd.read_csv(
    "D:/metcouncil_network_rebuild/data/processed/version_00/standard_networks/new_network_node_id.csv"
)

In [10]:
node_id_crosswalk = dict(zip(node_id_crosswalk_df.model_node_id_x, 
                             node_id_crosswalk_df.model_node_id_y))

In [4]:
# read the external records created by Rachel

In [5]:
input_externals_file = "D:/metcouncil_network_rebuild/data/external/taz/externals.csv"

In [6]:
external_connectors_df = pd.read_csv(input_externals_file)

In [7]:
external_connectors_df

,A,B
0,3031,233751
1,233751,3031
2,3032,40195
3,396497,3032
4,3033,225356
...,...,...
57,46819,3059
58,3060,219326
59,268251,3060
60,3061,16426


In [8]:
external_connectors_df.columns

Index(['A', 'B'], dtype='object')

In [11]:
external_connectors_df['A'] = external_connectors_df['A'].apply(
    lambda x: node_id_crosswalk.get(x) if x in node_id_crosswalk.keys() else x
)

external_connectors_df['B'] = external_connectors_df['B'].apply(
    lambda x: node_id_crosswalk.get(x) if x in node_id_crosswalk.keys() else x
)

In [12]:
external_connectors_df

,A,B
0,3031,108517
1,108517,3031
2,3032,211200
3,211199,3032
4,3033,180945
...,...,...
57,58648,3059
58,3060,272499
59,142548,3060
60,3061,108530


In [13]:
roadway_link_id_max = 1061656

In [16]:
external_connectors_df['lanes'] = 1
external_connectors_df['assign_group'] = 9
external_connectors_df['roadway_class'] = 99
external_connectors_df['centroidconnect'] = 1
external_connectors_df['bike_access'] = 1
external_connectors_df['drive_access'] = 1
external_connectors_df['walk_access'] = 1
external_connectors_df['roadway'] = 'taz'
external_connectors_df['rail_only'] = 0

external_connectors_df['model_link_id'] = range(
    roadway_link_id_max+1, 
    roadway_link_id_max+1+len(external_connectors_df)
)

In [17]:
external_connectors = external_connectors_df.to_dict(orient='records')

with open('external_connectors.yaml', "w") as outfile:
    yaml.dump(external_connectors, outfile, default_flow_style=False, sort_keys=False)

In [37]:
external_nodes_df = gpd.read_file(
    "D:/metcouncil_network_rebuild/data/external/taz/externals.shp"
)

In [38]:
external_nodes_df = external_nodes_df[['N', 'Lat', "Long"]]
external_nodes_df.rename(
    columns = {'N' : 'model_node_id', 'Lat' : 'Y', 'Long':'X'},
    inplace=True
)

In [39]:
external_nodes_df['bike_access'] = 1
external_nodes_df['drive_access'] = 1
external_nodes_df['walk_access'] = 1
external_nodes_df['rail_only'] = 0

In [40]:
external_nodes_df

,model_node_id,Y,X,bike_access,drive_access,walk_access
0,3031,45.740624,-93.230581,1,1,1
1,3032,45.743632,-92.991891,1,1,1
2,3033,45.654572,-92.673804,1,1,1
3,3034,45.743815,-92.378461,1,1,1
4,3035,45.566273,-92.128952,1,1,1
5,3036,45.394161,-92.110198,1,1,1
6,3037,45.134035,-92.132006,1,1,1
7,3038,44.919182,-92.113012,1,1,1
8,3039,44.879367,-92.118169,1,1,1
9,3040,44.634448,-92.112835,1,1,1


In [41]:
external_nodes = external_nodes_df[
    ['model_node_id', 'X', 'Y', 'drive_access', 'bike_access', 'walk_access']
].to_dict(orient='records')

with open('external_nodes.yaml', "w") as outfile:
    yaml.dump(external_nodes, outfile, default_flow_style=False, sort_keys=False)